In [1]:
instance_uri = "https://api.development.sinopia.io/resource/690eb696-98e6-4fa9-aecc-6f6ef774f339"
#

In [2]:
from rdflib import Graph, Namespace, URIRef
from rdflib.namespace import RDF
from lxml import etree as ET
from alma_ns import alma_namespaces

In [3]:
instance_uri = URIRef(instance_uri)
work_uri = None
instance_graph = Graph()
work_graph = Graph()
instance_graph.parse(instance_uri)


<Graph identifier=N99e31403e7e849309db0d3a6b02e0da5 (<class 'rdflib.graph.Graph'>)>

In [4]:
 # Define the bf and bflc namespaces
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")
for prefix, url in alma_namespaces:
    work_graph.bind(prefix, url)
work_uri = instance_graph.value(
    subject=URIRef(instance_uri), predicate=bf.instanceOf
        )
work_uri = URIRef(work_uri)
# Explicitly state that work_uri is of type bf:Work
work_graph.add((work_uri, RDF.type, bf.Work))

# parse the work graph
work_graph.parse(work_uri)

# add the instance to the work graph
work_graph.add((work_uri, bf.hasInstance, URIRef(instance_uri)))

# serialize the work graph
bfwork_alma_xml = work_graph.serialize(format="pretty-xml", encoding="utf-8")
tree = ET.fromstring(bfwork_alma_xml)
print(ET.tostring(tree, pretty_print=True).decode("utf-8"))


<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <bf:Work rdf:about="https://api.development.sinopia.io/resource/c61118ac-f5ad-4d1c-99c2-efb8c28898bf">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Text"/>
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="f266a043753234e7aaff2db03d4e71960b1">
        <bf:mainTitle xml:lang="de">Johannes Gutenberg</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:contribution>
      <bflc:PrimaryContribution rdf:nodeID="f266a043753234e7aaff2db03d4e71960b2">
        <bf:agent rdf:resource="http://id.loc.gov/authorities/names/no98125554"/>
        <bf:role>
          <bf:Role rdf:nodeID="f266a043753234e7aaff2db03d4e71960b3">
            <bf:

In [5]:
 # serialize to xml
bfwork_alma_xml = work_graph.serialize(format="pretty-xml", encoding="utf-8")
tree = ET.fromstring(bfwork_alma_xml)
# apply xslt to normalize instance
xslt = ET.parse("normalize-work-sinopia2alma.xsl")
transform = ET.XSLT(xslt)
bfwork_alma_xml = transform(tree)
bfwork_alma_xml = ET.tostring(
        bfwork_alma_xml, pretty_print=True, encoding="utf-8"
        )
print(f"Saved BFWork description for {work_uri}.")
print(bfwork_alma_xml.decode("utf-8"))

Saved BFWork description for https://api.development.sinopia.io/resource/c61118ac-f5ad-4d1c-99c2-efb8c28898bf.
<bib>
  <record_format>lcbf_work</record_format>
  <suppress_from_publishing>false</suppress_from_publishing>
  <record>
    <rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
  <bf:Work rdf:about="https://api.development.sinopia.io/resource/c61118ac-f5ad-4d1c-99c2-efb8c28898bf" xmlns:bf="http://id.loc.gov/ontologies/bibframe/">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Text"/>
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="f266a043753234e7aaff2db03d4e71960b1">
        <bf:mainTitle xml:lang="de">Johannes Gutenberg</bf:mainTitle>
      </bf:Title>
    </bf:title>
   